In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## GPU Configuration and Imports <a class="anchor" id="GPU-Configuration-and-Imports"></a>

In [2]:
import os
if os.getenv("CUDA_VISIBLE_DEVICES") is None:
    gpu_num = 0 # Use "" to use the CPU
    os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpu_num}"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)
# Avoid warnings from TensorFlow
tf.get_logger().setLevel('ERROR')


In [3]:
gpus = tf.config.list_physical_devices('GPU')
print(gpus)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pickle
import time
import os
import pandas as pd
import datetime
import tqdm
import h5py

from tensorflow.keras import Model
from tensorflow.keras.layers import Layer, Conv2D, LayerNormalization, SeparableConv2D, Normalization, BatchNormalization
from tensorflow.nn import relu

## **Neural Receiver <a class="anchor" id="Neural-Receiver"></a>**

In [5]:
class ResidualBlock(Model):
    r"""
    This Keras layer implements a convolutional residual block made of two convolutional layers with ReLU activation, layer normalization, and a skip connection.
    The number of convolutional channels of the input must match the number of kernel of the convolutional layers ``num_conv_channel`` for the skip connection to work.

    Input
    ------
    : [batch size, num time samples, num subcarriers, num_conv_channel], tf.float
        Input of the layer

    Output
    -------
    : [batch size, num time samples, num subcarriers, num_conv_channel], tf.float
        Output of the layer
    """

    def build(self, input_shape):

        self._layer_norm_1 = LayerNormalization(axis=[-1,-2,-3])
        self._conv_1 = Conv2D(filters= 128,
                              kernel_size=[3,3],
                              padding='same',
                              activation=None)

        self._layer_norm_2 = LayerNormalization(axis=[-1,-2,-3])
        self._conv_2 = Conv2D(filters= 128,
                              kernel_size=[3,3],
                              padding='same',
                              activation=None)

    def call(self, inputs):
        z = self._layer_norm_1(inputs)
        z = relu(z)
        z = self._conv_1(z)
        z = self._layer_norm_2(z)
        z = relu(z)
        z = self._conv_2(z) # [batch size, num time samples, num subcarriers, num_channels]
        # Skip connection
        z = z + inputs

        return z

class CustomNeuralReceiver(Model):
    r"""
    Keras layer implementing a residual convolutional neural receiver.

    This neural receiver is fed with the post-DFT received samples, forming a resource grid of size num_of_symbols x fft_size, and computes LLRs on the transmitted coded bits.
    These LLRs can then be fed to an outer decoder to reconstruct the information bits.

    Input
    ------
    y_no: [batch size, num ofdm symbols, num subcarriers, 2*num rx antenna + 1], tf.float32
        Concatenated received samples and noise variance.
(
    y : [batch size, num rx antenna, num ofdm symbols, num subcarriers], tf.complex
        Received post-DFT samples.

    no : [batch size], tf.float32
        Noise variance. At training, a different noise variance value is sampled for each batch example.
)
    Output
    -------
    : [batch size, num ofdm symbols, num subcarriers, num_bits_per_symbol]
        LLRs on the transmitted bits.
    """

    def __init__(self, training = False):
        super(CustomNeuralReceiver, self).__init__()
        self._training = training

    def build(self, input_shape):

        # Input convolution
        self._input_conv = Conv2D(filters= 128,
                                  kernel_size=[3,3],
                                  padding='same',
                                  activation=None)
        # Residual blocks
        self._res_block_1 = ResidualBlock()
        self._res_block_2 = ResidualBlock()
        self._res_block_3 = ResidualBlock()
        self._res_block_4 = ResidualBlock()
        # Output conv
        self._output_conv = Conv2D(filters= 2,    # QPSK
                                   kernel_size=[3,3],
                                   padding='same',
                                   activation=None)

    def call(self, inputs):
        # Split inputs size into 4RB-batchs.
        if self._training == False:
            padding_size = (-inputs.shape[1] % 48)
            padded_input_size = inputs.shape[1]
            if(padding_size != 0):
              padded_input_size = inputs.shape[1] + padding_size
              inputs = tf.concat([inputs, inputs[:,:padding_size,]],axis=1)
            inputs = tf.reshape(inputs, [-1,48,14,18])

        # Input conv
        z = self._input_conv(inputs)
        # Residual blocks
        z = self._res_block_1(z)
        z = self._res_block_2(z)
        z = self._res_block_3(z)
        z = self._res_block_4(z)
        # Output conv
        z = self._output_conv(z)


        if self._training == False:
            z = tf.reshape(z, [-1,padded_input_size,14,2])
            if padding_size != 0:
              z = z[:,:-(padding_size),]
        z = tf.concat([z[...,0:3,:],z[...,4:11,:], z[...,12:14,:]],axis=-2)
        z = tf.transpose(z, perm=[0,2,1,3])
        z = tf.reshape(z, [z.shape[0],(z.shape[1]*z.shape[2]*z.shape[3])])
        return z

#**Load Data**

- Function to load and preprocess data (y - receiver data and c - label data)

In [6]:
!mkdir hdf5
!cp -r /content/drive/MyDrive/Pusch_data/dataset/parquet .
!cp -r /content/drive/MyDrive/Pusch_data/dataset/hdf5/4RB_dataset_186k_samples.hdf5 /content/hdf5/

In [7]:
def load_hdf5(parent_name, group_name):
    with h5py.File(f'{parent_name}.hdf5', "r") as f:
        b = f[f"{group_name}_b"][:]
        c = f[f"{group_name}_c"][:]
        y = f[f"{group_name}_y"][:]
        r = f[f"{group_name}_r"][:]
    return b, c, y, r

def load_pickle(parent_name, group_name):
    """Saves data to a pickle file."""
    def load_from_pickle(filename):
        with open(filename, "rb") as f:
            return pickle.load(f)

    b = load_from_pickle(f'{parent_name}/{group_name}.b.pkl')
    c = load_from_pickle(f'{parent_name}/{group_name}.c.pkl')
    y = load_from_pickle(f'{parent_name}/{group_name}.y.pkl')
    r = load_from_pickle(f'{parent_name}/{group_name}.r.pkl')

    return b, c, y, r

def data_loader(df, dir, saved_dataset='hdf5'):
    assert saved_dataset in ['hdf5', 'pickle'], "saved data set should be 'pickle' or 'hdf5'."
    assert df['nTBSize'].nunique() == 1, "Not all elements have the same TB size."
    # assert df['Dmrs_mask'].map(len).nunique() == 1, "Not all elements have the same number of Dmrs/Data symbols."

    for pusch_record in df.itertuples():
        data_filename = pusch_record.Data_filename
        data_dirname = pusch_record.Data_dirname
        esno_db = pusch_record.Esno_db
        index = pusch_record.index
        if saved_dataset == 'hdf5':
            b,c,y, r = load_hdf5(f'{dir}/{data_dirname}', data_filename)
        else:
            b,c,y, r = load_pickle(f'{dir}/{data_dirname}', data_filename)
        yield index, esno_db, c, y, b, r

def preprocessing(index, esno_db, c, y, b, r):
    y = tf.concat([tf.math.real(y), tf.math.imag(y)], axis = 0)
    y = tf.transpose(y, perm=[2,1,0])
    y = (y - tf.reduce_mean(y)) / tf.math.reduce_std(y)
    r = tf.concat([tf.math.real(r), tf.math.imag(r)], axis = 0)
    r = tf.transpose(r, perm=[2,1,0])

    y_r = tf.concat([y, r], axis = -1)
    return index, esno_db, c, y, b, r, y_r



In [10]:
import os
from concurrent.futures import ThreadPoolExecutor

In [11]:
def load_data_in_parallel(pusch_record, dir, saved_dataset):
    data_filename = pusch_record.Data_filename
    data_dirname = pusch_record.Data_dirname
    esno_db = pusch_record.Esno_db
    index = pusch_record.index

    data_path = os.path.join(dir, data_dirname)

    if saved_dataset == 'hdf5':
        b, c, y, r = load_hdf5(data_path, data_filename)
    else:
        b, c, y, r = load_pickle(data_path, data_filename)

    return index, esno_db, c, y, b, r

def data_loader(df, dir, saved_dataset='hdf5', max_workers=4):
    assert saved_dataset in ['hdf5', 'pickle'], "saved data set should be 'pickle' or 'hdf5'."
    assert df['nTBSize'].nunique() == 1, "Not all elements have the same TB size."

    # Precompute the directory path to avoid repeated operations
    data_dir = os.path.abspath(dir)

    # Use ThreadPoolExecutor to load data in parallel (for I/O-bound tasks)
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        results = executor.map(lambda pusch_record: load_data_in_parallel(pusch_record, data_dir, saved_dataset), df.itertuples())

    # Yield results from the executor
    for result in results:
        yield result

**Load Data into Dataset**

In [ ]:
# NUM_SAMPLE = 4

- Data training has same size [ 4 x 12 ] (4RB Data Resource Grid). Then split into smaller blocks corresponding to nRB before training.


(Below is doing 1RB training)

- Drag the PUSCH_Data drive to your drive then edit the path below:


1.   pickles_dir: Dir to data folder
2.   parquet_path: Dir to meta data folder
- See document for details




* **Setup dataset_dir**

In [12]:
dataset_dir = f'/content/'
pickles_dir = f'{dataset_dir}/pickle'
hdf5_dir = f'{dataset_dir}/hdf5'
parquet_dir = f'{dataset_dir}/parquet'

parquet_name = '4RB_dataset_186k_samples' #4RB

* **Read and split data_folder into train and test set**

In [13]:
df = pd.read_parquet(f'{parquet_dir}/{parquet_name}.parquet', engine="pyarrow")
df = df.reset_index()

split_rate = 0.95

df_low = df[df['Esno_db'] <= -5]
df_mid = df[(df['Esno_db'] <= 0) & (df['Esno_db'] > -5)]
df_high = df[df['Esno_db'] > -0]

NUM_SAMPLE_LOW = len(df_low)
NUM_SAMPLE_MID = len(df_mid)
NUM_SAMPLE_HIGH = len(df_high)

df_low = df_low.sample(frac=1)
df_mid = df_mid.sample(frac=1)
df_high = df_high.sample(frac=1)

test_df_low = df_low.iloc[int(NUM_SAMPLE_LOW*split_rate):]
train_df_low = df_low.iloc[:int(NUM_SAMPLE_LOW*split_rate)]

test_df_mid = df_mid.iloc[int(NUM_SAMPLE_MID*split_rate):]
train_df_mid = df_mid.iloc[:int(NUM_SAMPLE_MID*split_rate)]

test_df_high = df_high.iloc[int(NUM_SAMPLE_HIGH*split_rate):]
train_df_high = df_high.iloc[:int(NUM_SAMPLE_HIGH*split_rate)]

print(f"Low: Total Size: {len(df_low)} sample, Test Size: {len(test_df_low)} sample, Train Size: {len(train_df_low)} sample")
print(f"Mid: Total Size: {len(df_mid)} sample, Test Size: {len(test_df_mid)} sample, Train Size: {len(train_df_mid)} sample")
print(f"High: Total Size: {len(df_high)} sample, Test Size: {len(test_df_high)} sample, Train Size: {len(train_df_high)} sample")


Low: Total Size: 66044 sample, Test Size: 3303 sample, Train Size: 62741 sample
Mid: Total Size: 60040 sample, Test Size: 3002 sample, Train Size: 57038 sample
High: Total Size: 60040 sample, Test Size: 3002 sample, Train Size: 57038 sample


* **Load data into Dataset object**

In [14]:
def load_data(train_df, test_df, hdf5_dir, batch_train, batch_test):
  train_set = tf.data.Dataset.from_generator(
            lambda: data_loader(train_df, hdf5_dir),
            output_types=(tf.int32, tf.float32, tf.float32, tf.complex64, tf.float32, tf.complex64))

  train_set = train_set.cache()
  train_set = train_set.prefetch(tf.data.AUTOTUNE)
  train_set = train_set.map(preprocessing).batch(batch_train)

  # Load test_data into Dataset
  test_set = tf.data.Dataset.from_generator(
              lambda: data_loader(test_df, hdf5_dir),
              output_types=(tf.int32, tf.float32, tf.float32, tf.complex64, tf.float32, tf.complex64))

  test_set = test_set.cache()
  test_set = test_set.prefetch(tf.data.AUTOTUNE)
  test_set = test_set.map(preprocessing).batch(batch_test)
  return train_set, test_set


In [15]:
BATCH_TRAIN = 256
BATCH_TEST = 64

train_set_low, test_set_low = load_data(train_df_low, test_df_low, hdf5_dir, BATCH_TRAIN, BATCH_TEST)
train_set_mid, test_set_mid = load_data(train_df_mid, test_df_mid, hdf5_dir, BATCH_TRAIN, BATCH_TEST)
train_set_high, test_set_high = load_data(train_df_high, test_df_high, hdf5_dir, BATCH_TRAIN, BATCH_TEST)



In [16]:
# First load data for accelerating
for n, (index, esno_db, c, y, b, r, y_r) in enumerate(test_set_low):
    print(y.shape)

for n, (index, esno_db, c, y, b, r, y_r) in enumerate(test_set_mid):
    print(y.shape)

for n, (index, esno_db, c, y, b, r, y_r) in enumerate(test_set_high):
    print(y.shape)

for n, (index, esno_db, c, y, b, r, y_r) in enumerate(train_set_high):
    print(y.shape)


(64, 48, 14, 16)
(64, 48, 14, 16)
(64, 48, 14, 16)
(64, 48, 14, 16)
(64, 48, 14, 16)
(64, 48, 14, 16)
(64, 48, 14, 16)
(64, 48, 14, 16)
(64, 48, 14, 16)
(64, 48, 14, 16)
(64, 48, 14, 16)
(64, 48, 14, 16)
(64, 48, 14, 16)
(64, 48, 14, 16)
(64, 48, 14, 16)
(64, 48, 14, 16)
(64, 48, 14, 16)
(64, 48, 14, 16)
(64, 48, 14, 16)
(64, 48, 14, 16)
(64, 48, 14, 16)
(64, 48, 14, 16)
(64, 48, 14, 16)
(64, 48, 14, 16)
(64, 48, 14, 16)
(64, 48, 14, 16)
(64, 48, 14, 16)
(64, 48, 14, 16)
(64, 48, 14, 16)
(64, 48, 14, 16)
(64, 48, 14, 16)
(64, 48, 14, 16)
(64, 48, 14, 16)
(64, 48, 14, 16)
(64, 48, 14, 16)
(64, 48, 14, 16)
(64, 48, 14, 16)
(64, 48, 14, 16)
(64, 48, 14, 16)
(64, 48, 14, 16)
(64, 48, 14, 16)
(64, 48, 14, 16)
(64, 48, 14, 16)
(64, 48, 14, 16)
(64, 48, 14, 16)
(64, 48, 14, 16)
(64, 48, 14, 16)
(64, 48, 14, 16)
(64, 48, 14, 16)
(64, 48, 14, 16)
(64, 48, 14, 16)
(39, 48, 14, 16)
(64, 48, 14, 16)
(64, 48, 14, 16)
(64, 48, 14, 16)
(64, 48, 14, 16)
(64, 48, 14, 16)
(64, 48, 14, 16)
(64, 48, 14, 1

In [ ]:
# model._training = False
# llr = model(y_r)
# loss = loss_cal(llr, c)
# print(loss)

tf.Tensor(0.12375993, shape=(), dtype=float32)


- Checking size of a sample data in Dataset
- Preprocessing is called when load Data

#**Training Model**



*   Creat Model and Optimizer Instance
*   Set mode of "pretrained" ( True to load pretrained weights if exist )

In [17]:
model = CustomNeuralReceiver(training = True)
inputs = tf.zeros([1,48,14,18])
model(inputs)
model.summary()

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

pretrained = False

Model: "custom_neural_receiver"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (1, 48, 14, 128)            │          20,864 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ residual_block (ResidualBlock)       │ ?                           │         639,232 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ residual_block_1 (ResidualBlock)     │ ?                           │         639,232 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ residual_block_2 (ResidualBlock)     │ ?                           │         639,232 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ residual_block_3 (ResidualBlock)     │ ?                           │         639,232 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (1, 48, 14, 2)              │           2,306 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,580,098 (9.84 MB)

 Trainable params: 2,580,098 (9.84 MB)

 Non-trainable params: 0 (0.00 B)

In [18]:
# Define our metrics
train_batch_loss = tf.keras.metrics.Mean('Epoch_train_loss', dtype=tf.float32)

val_batch_loss_low = tf.keras.metrics.Mean('Epoch_val_loss_low', dtype=tf.float32)
val_batch_loss_mid = tf.keras.metrics.Mean('Epoch_val_loss_mid', dtype=tf.float32)
val_batch_loss_high = tf.keras.metrics.Mean('Epoch_val_loss_high', dtype=tf.float32)


- **Define Train_step and Loss_cal**





In [19]:
def loss_cal(pred, labels):
  bce = tf.nn.sigmoid_cross_entropy_with_logits(labels, pred)
  bce = tf.reduce_mean(bce)
  loss = bce
  return loss


#train model with nRB
@tf.function
def train_step(inputs, labels):
  with tf.GradientTape() as tape:
    llr = model(inputs)
    loss = loss_cal(llr, labels)
  weights = model.trainable_weights
  grads = tape.gradient(loss, weights)
  optimizer.apply_gradients(zip(grads, weights))
  # train_accuracy(labels, llr)
  return loss

#test model with nRB
@tf.function
def val_step(inputs, labels):
  llr = model(inputs)
  loss = loss_cal(llr, labels)
  # train_accuracy(labels, llr)
  return loss

- **Load and save pretrained Model**

In [20]:
def load_weights(model, pretrained_weights_path):
    # Build Model with random input
    # Load weights
  with open(pretrained_weights_path, 'rb') as f:
    weights = pickle.load(f)
    model.set_weights(weights)
    print(f"Loaded pretrained weights from {pretrained_weights_path}")

def save_weights(model, model_folder_path, model_file_name):
    # Save the weights in a file
    model_weights_path = os.path.join(model_folder_path, model_file_name)
    weights = model.get_weights()
    with open(model_weights_path, 'wb') as f:
        pickle.dump(weights, f)

def save_trainable_weights(model, model_folder_path, model_file_name):
    # Save the weights in a file
    model_weights_path = os.path.join(model_folder_path, model_file_name)
    weights = model.trainable_weights
    with open(model_weights_path, 'wb') as f:
        pickle.dump(weights, f)

In [ ]:
if pretrained:
  pretrained_weights_path = '/content/drive/MyDrive/AI_for_PUSCH/VHT_neural_receiver/weight_4RB_normalize_with_DMRS_org.pkl'
  load_weights(model, pretrained_weights_path)

- **Training model**

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [21]:
#Initialize folder log for tensorboard
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

#Setup log_path
train_log_dir = '/content/drive/MyDrive/AI_for_PUSCH/logs/gradient_tape/' + current_time + '/train'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)

val_log_dir = '/content/drive/MyDrive/AI_for_PUSCH/logs/gradient_tape/' + current_time + '/val'
val_summary_writer = tf.summary.create_file_writer(val_log_dir)

* ***If using VSCode, run "tensorboard --logdir /log_path" in terminal***

In [ ]:
%tensorboard --logdir /content/drive/MyDrive/AI_for_PUSCH/logs/gradient_tape/



*   **Define global params for training**



In [22]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
# Define num EPOCHS, Model_path, Model_name to save model during training
EPOCHS = 1000
SAVE_AFTER_NUM_EPOCH = 2
model_folder_path = "/content/drive/MyDrive/AI_for_PUSCH/VHT_neural_receiver/"    #Custom here

In [ ]:
if not pretrained:
  pre_batch_loss = 1e9
  for epoch in range(EPOCHS):
      start_time = time.time()
      batch_loss = np.array([])
      batch_val_loss_low = np.array([])
      batch_val_loss_mid = np.array([])
      batch_val_loss_high = np.array([])
      print(
        f'Epoch {epoch}, '
      )

    # Load batch data for 4RB training
      for iter, (index, esno_db, c, y, b,r, y_r) in enumerate(train_set_low):
          # calculate loss and optimize
          loss = train_step(y_r, c)
          batch_loss = np.append(batch_loss, loss)


      # Load batch data for 4RB validating
      for iter, (index, esno_db, c, y, b,r, y_r) in enumerate(test_set_low):
          loss_val = val_step(y_r, c)
          batch_val_loss_low = np.append(batch_val_loss_low, loss_val)

      for iter, (index, esno_db, c, y, b,r, y_r) in enumerate(test_set_mid):
          loss_val = val_step(y_r, c)
          batch_val_loss_mid = np.append(batch_val_loss_mid, loss_val)

      for iter, (index, esno_db, c, y, b,r, y_r) in enumerate(test_set_high):
          loss_val = val_step(y_r, c)
          batch_val_loss_high = np.append(batch_val_loss_high, loss_val)


      # Write batch loss to log file
      train_batch_loss(tf.reduce_mean(batch_loss))
      with train_summary_writer.as_default():
        tf.summary.scalar('BCE_Epoch_Loss', train_batch_loss.result(), step=epoch+134)

      val_batch_loss_low(tf.reduce_mean(batch_val_loss_low))
      with val_summary_writer.as_default():
        tf.summary.scalar('BCE_Epoch_Loss', val_batch_loss_low.result(), step=epoch+134)

      val_batch_loss_mid(tf.reduce_mean(batch_val_loss_mid))
      with val_summary_writer.as_default():
        tf.summary.scalar('BCE_Epoch_Loss', val_batch_loss_mid.result(), step=epoch+134)

      val_batch_loss_high(tf.reduce_mean(batch_val_loss_high))
      with val_summary_writer.as_default():
        tf.summary.scalar('BCE_Epoch_Loss', val_batch_loss_high.result(), step=epoch+134)

      time_taken = time.time() - start_time
      print(
          f'Epoch {epoch}, '
          f'Train_Loss: {train_batch_loss.result():0.4f}, '
          f'Val_Loss_low: {val_batch_loss_low.result():0.4f}, '
          f'Val_Loss_mid: {val_batch_loss_mid.result():0.4f}, '
          f'Val_Loss_high: {val_batch_loss_high.result():0.4f}, '
          f'Time taken: {time_taken:0.2f}'
      )

      if pre_batch_loss >= val_batch_loss_high.result():
        pre_batch_loss = val_batch_loss_high.result()
        model_file_name = f"weight_4RB_high_SNR_dynamic_config.pkl"
        save_weights(model, model_folder_path, model_file_name)
        print(
            f'Save model at epoch {epoch}'
        )

      train_batch_loss.reset_state()
      val_batch_loss_low.reset_state()
      val_batch_loss_mid.reset_state()
      val_batch_loss_high.reset_state()


Epoch 0, 
Epoch 0, Train_Loss: 0.6900, Val_Loss_low: 0.6830, Val_Loss_mid: 0.6504, Val_Loss_high: 0.6223, Time taken: 65.29
Save model at epoch 0
Epoch 1, 
Epoch 1, Train_Loss: 0.6482, Val_Loss_low: 0.6350, Val_Loss_mid: 0.5261, Val_Loss_high: 0.4255, Time taken: 65.10
Save model at epoch 1
Epoch 2, 
Epoch 2, Train_Loss: 0.6216, Val_Loss_low: 0.6206, Val_Loss_mid: 0.4985, Val_Loss_high: 0.4051, Time taken: 65.15
Save model at epoch 2
Epoch 3, 
Epoch 3, Train_Loss: 0.5964, Val_Loss_low: 0.6010, Val_Loss_mid: 0.4629, Val_Loss_high: 0.3548, Time taken: 65.13
Save model at epoch 3
Epoch 4, 
Epoch 4, Train_Loss: 0.5718, Val_Loss_low: 0.5675, Val_Loss_mid: 0.3982, Val_Loss_high: 0.2605, Time taken: 65.16
Save model at epoch 4
Epoch 5, 
Epoch 5, Train_Loss: 0.5602, Val_Loss_low: 0.5758, Val_Loss_mid: 0.3938, Val_Loss_high: 0.2501, Time taken: 65.15
Save model at epoch 5
Epoch 6, 
Epoch 6, Train_Loss: 0.5417, Val_Loss_low: 0.5689, Val_Loss_mid: 0.3756, Val_Loss_high: 0.2413, Time taken: 65.16


* **Save weight**

In [ ]:
    # Save the weights in a file
model_folder_path = "/content/drive/MyDrive/AI_for_PUSCH/VHT_neural_receiver/"
model_file_name = "weight_4RB_UMI_dynamic_config.pkl"
save_weights(model, model_folder_path, model_file_name)